In [1]:
import sys
sys.path.append('E:/Data Science/ML/ud120-projects/tools/')
sys.path.append('E:/Data Science/ML/ud120-projects/choose_your_own')
sys.path.append('E:/Data Science/ML/ud120-projects/datasets_questions')

import os
os.chdir('E:/Data Science/ML/ud120-projects/feature_selection')

In [2]:
import pickle
import numpy
numpy.random.seed(42)

import warnings
warnings.simplefilter('ignore')


words_file = 'E:/Data Science/ML/ud120-projects/feature_selection/word_data_overfit.pkl' 
authors_file = 'E:/Data Science/ML/ud120-projects/feature_selection/email_authors_overfit.pkl'
    
def my_func(words_file, authors_file):
    '''
    I will use this code later in the lesson so I made it a function
    '''
    
    ### The words (features) and authors (labels), already largely processed.
    ### These files should have been created from the previous (Lesson 10)
    ### mini-project.
    word_data = pickle.load( open(words_file, 'r'))
    authors = pickle.load( open(authors_file, 'r') )



    ### test_size is the percentage of events assigned to the test set (the
    ### remainder go into training)
    ### feature matrices changed to dense representations for compatibility with
    ### classifier functions in versions 0.15.2 and earlier
    from sklearn import cross_validation
    features_train, features_test, labels_train, labels_test = cross_validation.train_test_split(word_data, authors, test_size=0.1, random_state=42)

    from sklearn.feature_extraction.text import TfidfVectorizer
    vectorizer = TfidfVectorizer(sublinear_tf=True, max_df=0.5,
                                 stop_words='english')
    features_train = vectorizer.fit_transform(features_train)
    features_test  = vectorizer.transform(features_test).toarray()


    ### a classic way to overfit is to use a small number
    ### of data points and a large number of features;
    ### train on only 150 events to put ourselves in this regime
    features_train = features_train[:150].toarray()
    labels_train   = labels_train[:150]


    from sklearn.tree import DecisionTreeClassifier

    clf = DecisionTreeClassifier()
    clf.fit(features_train, labels_train)
    
    return clf, vectorizer, features_train, features_test, labels_train, labels_test


(clf, vectorizer, features_train, features_test, labels_train, labels_test) = my_func(words_file, authors_file)

print 'Number of training points = {0}'.format(len(features_train))

D:\Python\Anaconda2\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


Number of training points = 150


# Accuracy of Your Overfit Decision Tree

In [3]:
clf.score(features_test, labels_test)

0.95904436860068254

# Identify the Most Powerful Features and Use TfIdf to Get the Most Important Word

In [4]:
top_features = [(number, feature, vectorizer.get_feature_names()[number]) for number, feature in 
                zip(range(len(clf.feature_importances_)), clf.feature_importances_) if feature > 0.2]
top_features

[(33604, 0.76470588235294124, u'sshacklensf')]